<a href="https://colab.research.google.com/github/AntonyZhuang/Capstone_GeneticAlgorithm/blob/main/GA_knapsack_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import random

class GeneticAlgorithm:
    def __init__(self, population_size, chromosome_length, crossover_rate, mutation_rate,
                 max_generations, tournament_size, fitness_fn, penalty_mode=False,
                 penalty_factor=5, early_stop_generations=30):
        self.population_size = population_size
        self.chromosome_length = chromosome_length
        self.crossover_rate = crossover_rate
        self.base_mutation_rate = mutation_rate
        self.max_generations = max_generations
        self.tournament_size = tournament_size
        self.fitness_fn = fitness_fn
        self.penalty_mode = penalty_mode
        self.penalty_factor = penalty_factor
        self.early_stop_generations = early_stop_generations

    def initialize_population(self):
        return [[random.randint(0, 1) for _ in range(self.chromosome_length)]
                for _ in range(self.population_size)]

    def evaluate_population(self, population):
        # Fitness calculation (with penalty mode toggle)
        fitnesses = [self.fitness_fn(ind, penalty_mode=self.penalty_mode,
                                     penalty_factor=self.penalty_factor) for ind in population]
        return fitnesses

    def tournament_selection(self, population, fitnesses):
        best = None
        for _ in range(self.tournament_size):
            idx = random.randint(0, len(population) - 1)
            if best is None or fitnesses[idx] > fitnesses[best]:
                best = idx
        return population[best]

    def crossover(self, parent1, parent2):
        if random.random() < self.crossover_rate:
            point1 = random.randint(1, self.chromosome_length - 2)
            point2 = random.randint(point1 + 1, self.chromosome_length - 1)
            child1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
            child2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]
            return child1, child2
        return parent1[:], parent2[:]

    def mutate(self, individual, mutation_rate):
        for i in range(self.chromosome_length):
            if random.random() < mutation_rate:
                individual[i] = 1 - individual[i]
        return individual

    def run(self):
        population = self.initialize_population()
        fitnesses = self.evaluate_population(population)

        best_solution = population[fitnesses.index(max(fitnesses))]
        best_fitness = max(fitnesses)

        no_improvement = 0
        mutation_rate = self.base_mutation_rate
        history = []

        for generation in range(self.max_generations):
            new_population = []

            # Elitism: keep top 2
            sorted_indices = sorted(range(len(fitnesses)), key=lambda i: fitnesses[i], reverse=True)
            elite = [population[sorted_indices[0]], population[sorted_indices[1]]]
            new_population.extend([elite[0][:], elite[1][:]])

            # Generate new individuals
            while len(new_population) < self.population_size:
                parent1 = self.tournament_selection(population, fitnesses)
                parent2 = self.tournament_selection(population, fitnesses)
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1, mutation_rate)
                child2 = self.mutate(child2, mutation_rate)
                new_population.extend([child1, child2])

            population = new_population[:self.population_size]
            fitnesses = self.evaluate_population(population)

            gen_best = max(fitnesses)
            gen_avg = sum(fitnesses) / len(fitnesses)
            history.append((generation + 1, gen_best, gen_avg))

            # Track improvement
            if gen_best > best_fitness:
                best_fitness = gen_best
                best_solution = population[fitnesses.index(gen_best)]
                no_improvement = 0
                mutation_rate = self.base_mutation_rate  # reset mutation
            else:
                no_improvement += 1
                # Adaptive mutation: increase slightly if no improvement
                mutation_rate = min(0.1, mutation_rate * 1.05)

            print(f"Gen {generation+1}: Best = {gen_best} | Avg = {gen_avg:.2f} | Mutation = {mutation_rate:.3f}")

            # Early stopping if stagnant
            if no_improvement >= self.early_stop_generations:
                print(f"\nEarly stopping at generation {generation+1} (no improvement in {self.early_stop_generations} gens)")
                break

        return best_solution, best_fitness, history


In [46]:
import random

# Fixed random seed for reproducibility
SEED = 42
random.seed(SEED)

# Problem constants
CHROMOSOME_LENGTH = 128

# Generate random values and weights
VALUES = [random.randint(10, 100) for _ in range(CHROMOSOME_LENGTH)]
WEIGHTS = [random.randint(5, 50) for _ in range(CHROMOSOME_LENGTH)]

# Hardcoded capacity = 40% of total weight
CAPACITY = int(sum(WEIGHTS) * 0.4)

def knapsack_fitness(individual, penalty_mode=False, penalty_factor=5):
    total_value = 0
    total_weight = 0
    for bit, value, weight in zip(individual, VALUES, WEIGHTS):
        if bit == 1:
            total_value += value
            total_weight += weight
    if not penalty_mode:
        return total_value if total_weight <= CAPACITY else 0
    else:
        # Penalize overweight instead of discarding
        penalty = penalty_factor * max(0, total_weight - CAPACITY)
        return total_value - penalty


In [47]:
ga = GeneticAlgorithm(
        population_size=100,
        chromosome_length=CHROMOSOME_LENGTH,
        crossover_rate=0.8,
        mutation_rate=0.01,
        max_generations=200,
        tournament_size=3,
        fitness_fn=knapsack_fitness,
        penalty_mode=True,          # Use penalty mode
        penalty_factor=5,           # Penalty weight
        early_stop_generations=70   # Stop if stagnant
)

best_solution, best_fitness, history = ga.run()

print("\nFinal Best Solution:")
print("Best Fitness (Value):", best_fitness)
print("Binary Solution:", "".join(map(str, best_solution)))

Gen 1: Best = 3437 | Avg = 2330.34 | Mutation = 0.010
Gen 2: Best = 3437 | Avg = 2677.19 | Mutation = 0.011
Gen 3: Best = 3636 | Avg = 2957.61 | Mutation = 0.010
Gen 4: Best = 3636 | Avg = 3151.81 | Mutation = 0.011
Gen 5: Best = 3754 | Avg = 3298.47 | Mutation = 0.010
Gen 6: Best = 3754 | Avg = 3401.30 | Mutation = 0.011
Gen 7: Best = 3754 | Avg = 3483.75 | Mutation = 0.011
Gen 8: Best = 3792 | Avg = 3523.30 | Mutation = 0.010
Gen 9: Best = 3860 | Avg = 3551.23 | Mutation = 0.010
Gen 10: Best = 3860 | Avg = 3627.14 | Mutation = 0.011
Gen 11: Best = 3942 | Avg = 3640.95 | Mutation = 0.010
Gen 12: Best = 3942 | Avg = 3680.27 | Mutation = 0.011
Gen 13: Best = 4015 | Avg = 3718.17 | Mutation = 0.010
Gen 14: Best = 4015 | Avg = 3755.89 | Mutation = 0.011
Gen 15: Best = 4015 | Avg = 3772.22 | Mutation = 0.011
Gen 16: Best = 4052 | Avg = 3802.64 | Mutation = 0.010
Gen 17: Best = 4074 | Avg = 3854.91 | Mutation = 0.010
Gen 18: Best = 4084 | Avg = 3863.27 | Mutation = 0.010
Gen 19: Best = 4136

In [ ]:
def knapsack_dp(values, weights, capacity):
    n = len(values)
    dp = [0] * (capacity + 1)
    for i in range(n):
        w, v = weights[i], values[i]
        for c in range(capacity, w - 1, -1):
            dp[c] = max(dp[c], dp[c - w] + v)
    return dp[capacity]

best_value = knapsack_dp(VALUES, WEIGHTS, CAPACITY)
print("Optimal value (via DP):", best_value)